In [1]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.__version__

'1.15.5'

In [2]:
PATH = os.getcwd()

LOG_DIR = PATH+ '/embedding-logs'
#metadata = os.path.join(LOG_DIR, 'metadata2.tsv')

In [3]:
data_path = '/home/ec2-user/SageMaker/wikiart_styles_224'
data_dir_list = os.listdir(data_path)

In [4]:
img_data=[]
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(224,224))
        img_data.append(input_img_resize)
    
                
img_data = np.array(img_data)

Loaded the images of dataset-Pointillism

Loaded the images of dataset-Contemporary_Realism

Loaded the images of dataset-High_Renaissance

Loaded the images of dataset-Cubism

Loaded the images of dataset-Post_Impressionism

Loaded the images of dataset-Impressionism

Loaded the images of dataset-Fauvism

Loaded the images of dataset-Minimalism

Loaded the images of dataset-Pop_Art

Loaded the images of dataset-Expressionism

Loaded the images of dataset-Baroque

Loaded the images of dataset-Abstract_Expressionism

Loaded the images of dataset-Realism

Loaded the images of dataset-Naive_Art_Primitivism



MemoryError: Unable to allocate 7.96 GiB for an array with shape (56800, 224, 224, 3) and data type uint8

In [5]:
len(img_data)

56800

In [6]:
feature_vectors = np.loadtxt('feature_vectors_400_samples.txt')
print ("feature_vectors_shape:",feature_vectors.shape)
print ("num of images:",feature_vectors.shape[0])
print ("size of individual feature vector:",feature_vectors.shape[1])

feature_vectors_shape: (400, 4096)
num of images: 400
size of individual feature vector: 4096


In [7]:
num_of_samples=feature_vectors.shape[0]
num_of_samples_each_class = 100

In [8]:
features = tf.Variable(feature_vectors, name='features')

In [9]:
y = np.ones((num_of_samples,),dtype='int64')

y[0:100]=0
y[100:200]=1
y[200:300]=2
y[300:]=3

names = ['cats','dogs','horses','humans']

In [10]:
import os, argparse
import pickle
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
from sklearn.externals import joblib
from keras import backend as K
import keras

ModuleNotFoundError: No module named 'spacy'

In [ ]:

    
# File paths for the model, all of these except the CNN Weights are 
# provided in the repo, See the VGG_model/README.md to download VGG weights
CNN_weights_file_name   = 'VGG_model/vgg16_weights.h5'


# Chagne the value of verbose to 0 to avoid printing the progress statements
verbose = 1

def get_image_model(CNN_weights_file_name):
    ''' Takes the CNN weights file, and returns the VGG model update 
    with the weights. Requires the file VGG.py inside models/CNN '''
    from VGG_model.VGG import VGG_16
    image_model = VGG_16(CNN_weights_file_name)

    # this is standard VGG 16 without the last two layers
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    # one may experiment with "adam" optimizer, but the loss function for
    # this kind of task is pretty standard
    image_model.compile(optimizer=sgd, loss='categorical_crossentropy')
    return image_model

vgg16_model = get_image_model(CNN_weights_file_name)

def get_image_features(image_file_name):
    ''' Runs the given image_file to VGG 16 model and returns the 
    weights (filters) as a 1, 4096 dimension vector '''
    image_features = np.zeros((1, 4096))
    # Magic_Number = 4096  > Comes from last layer of VGG Model

    # Since VGG was trained as a image of 224x224, every new image
    # is required to go through the same transformation
    im = cv2.resize(cv2.imread(image_file_name), (224, 224))


    # The mean pixel values are taken from the VGG authors, which are the values computed from the training dataset.
    mean_pixel = [103.939, 116.779, 123.68]

    im = im.astype(np.float32, copy=False) # shape of im = (224,224,3)
    
    for c in range(3):
        im[:, :, c] = im[:, :, c] - mean_pixel[c]        

    im = im.transpose((2,0,1)) # convert the image to RGBA  # shame of im= (3,224,224)

    
    # this axis dimension is required becuase VGG was trained on a dimension
    # of 1, 3, 224, 224 (first axis is for the batch size
    # even though we are using only one image, we have to keep the dimensions consistent
    im = np.expand_dims(im, axis=0)  # shape of im = (1,3,224,224)

    image_features[0,:] = vgg16_model.predict(im)[0]
    return image_features


PATH=os.getcwd()
data_path = PATH + '/data'
data_dir_list = os.listdir(data_path)

image_features_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Extracting Features of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        image_features=get_image_features(data_path + '/'+ dataset + '/'+ img )
        image_features_list.append(image_features)
    
    
image_features_arr=np.asarray(image_features_list)
image_features_arr = np.rollaxis(image_features_arr,1,0)
image_features_arr = image_features_arr[0,:,:]

np.savetxt('feature_vectors_400_samples.txt',image_features_arr)
#feature_vectors = np.loadtxt('feature_vectors.txt')
pickle.dump(image_features_arr, open('feature_vectors_400_samples.pkl', 'wb'))